# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_vacation_df=pd.read_csv("output_data/cities.csv", encoding="utf-8")
city_vacation_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,546105,Nikolskoye,59.70,30.79,33.80,80,90,8.95,RU
1,1,3833367,Ushuaia,-54.80,-68.30,46.40,45,20,31.09,AR
2,2,4030556,Rikitea,-23.12,-134.97,76.08,79,24,7.52,PF
3,3,6170031,Tuktoyaktuk,69.45,-133.04,8.60,84,90,19.46,CA
4,4,5122534,Jamestown,42.10,-79.24,39.99,60,90,26.40,US


In [3]:
city_vacation_df = city_vacation_df.dropna()
city_vacation_df.count()

Unnamed: 0    540
City ID       540
City          540
Lat           540
Long          540
Max Temp      540
Humidity      540
Cloudiness    540
Wind Speed    540
Country       540
dtype: int64

In [4]:
city_vacation_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,546105,Nikolskoye,59.70,30.79,33.80,80,90,8.95,RU
1,1,3833367,Ushuaia,-54.80,-68.30,46.40,45,20,31.09,AR
2,2,4030556,Rikitea,-23.12,-134.97,76.08,79,24,7.52,PF
3,3,6170031,Tuktoyaktuk,69.45,-133.04,8.60,84,90,19.46,CA
4,4,5122534,Jamestown,42.10,-79.24,39.99,60,90,26.40,US


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

# Create coordinates
coordinates = city_vacation_df[['Lat','Long']].astype(float)

#Humidity as weight
humidity = city_vacation_df['Humidity'].astype(float)

In [6]:
# Create a humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)

fig.add_layer(heat_layer)

#diaplay map 
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
#Make copy of dataframe
city_vacation_df.copy().head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
0,0,546105,Nikolskoye,59.70,30.79,33.80,80,90,8.95,RU
1,1,3833367,Ushuaia,-54.80,-68.30,46.40,45,20,31.09,AR
2,2,4030556,Rikitea,-23.12,-134.97,76.08,79,24,7.52,PF
3,3,6170031,Tuktoyaktuk,69.45,-133.04,8.60,84,90,19.46,CA
4,4,5122534,Jamestown,42.10,-79.24,39.99,60,90,26.40,US


In [8]:
# NO COLD PLACES MAX TEMPS ABOVE 80F
vacation_spots = city_vacation_df.loc[city_vacation_df['Max Temp']>= 80]
vacation_spots.count()

Unnamed: 0    75
City ID       75
City          75
Lat           75
Long          75
Max Temp      75
Humidity      75
Cloudiness    75
Wind Speed    75
Country       75
dtype: int64

In [9]:
# NOT CLOUDY
vacation_spots = vacation_spots.loc[vacation_spots['Cloudiness']<= 40]
vacation_spots.count()

Unnamed: 0    40
City ID       40
City          40
Lat           40
Long          40
Max Temp      40
Humidity      40
Cloudiness    40
Wind Speed    40
Country       40
dtype: int64

In [10]:
# No to humid, but no rain
vacation_spots = vacation_spots.loc[vacation_spots['Humidity']<= 50]
vacation_spots.count()

Unnamed: 0    10
City ID       10
City          10
Lat           10
Long          10
Max Temp      10
Humidity      10
Cloudiness    10
Wind Speed    10
Country       10
dtype: int64

In [11]:
hotel_df = vacation_spots

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [12]:
#Alwayds making sure what my data looks like
hotel_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country
20,20,1651810,Airai,-8.93,125.41,82.78,37,2,7.18,TL
61,61,1257771,Saiha,22.48,92.97,85.26,40,0,1.57,IN
100,100,2167426,Emerald,-23.53,148.17,100.99,19,0,10.29,AU
126,126,1254709,Tezu,27.92,96.17,81.01,31,0,3.06,IN
216,216,2070998,Geraldton,-28.77,114.60,82.40,23,0,5.82,AU


In [13]:
#cleaning and organizing hotel_df to add new cloumn Hotel Name types: lodging
hotel_df.drop(['Unnamed: 0', 'City ID'], axis=1)
hotel_df['Hotel Name'] = ''
hotel_df.head()

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
20,20,1651810,Airai,-8.93,125.41,82.78,37,2,7.18,TL,
61,61,1257771,Saiha,22.48,92.97,85.26,40,0,1.57,IN,
100,100,2167426,Emerald,-23.53,148.17,100.99,19,0,10.29,AU,
126,126,1254709,Tezu,27.92,96.17,81.01,31,0,3.06,IN,
216,216,2070998,Geraldton,-28.77,114.60,82.40,23,0,5.82,AU,


In [14]:
#Building google Api, parameters, types: lodging  keyword: hotel

params = {"types":"lodging", "keyword": "hotel", 'radius': 5000, 'key': g_key}

for index, row in hotel_df.iterrows():
    
    lat = row["Lat"]
    long = row["Long"]
    params['location'] =f"{lat},{long}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

    
    response = requests.get(base_url, params=params)
    print(response)
    
    hotel_names = response.json() 
    
#     print(json.dumps(hotel_names, indent=4, sort_keys=True)
#     print(hotel_names["results"][0]['name'])
# Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_names["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    

<Response [200]>
Missing field/result... skipping.
<Response [200]>
Missing field/result... skipping.
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>
<Response [200]>


In [15]:
hotel_df

,Unnamed: 0,City ID,City,Lat,Long,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Hotel Name
20,20,1651810,Airai,-8.93,125.41,82.78,37,2,7.18,TL,
61,61,1257771,Saiha,22.48,92.97,85.26,40,0,1.57,IN,
100,100,2167426,Emerald,-23.53,148.17,100.99,19,0,10.29,AU,Direct Hotels - Villas on Rivergum
126,126,1254709,Tezu,27.92,96.17,81.01,31,0,3.06,IN,Capital O 66991 Hotel Highway
216,216,2070998,Geraldton,-28.77,114.60,82.40,23,0,5.82,AU,Ocean Centre Hotel
248,248,1275637,Bilāspur,22.08,82.15,80.31,50,0,3.98,IN,Courtyard by Marriott Bilaspur
269,269,1272962,Dhanera,24.52,72.02,80.24,31,0,3.96,IN,Aashapura Hotal
420,420,2177233,Bairnsdale,-37.83,147.62,81.00,34,2,7.00,AU,Bairnsdale Motel
462,462,2155562,Nelson Bay,-32.72,152.15,98.60,23,0,12.75,AU,Amarna Luxury Beach Resort - Guest Only
473,473,2171722,Charters Towers,-20.10,146.27,98.29,20,0,16.78,AU,Commercial Hotel


In [16]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Long"]]

In [21]:
# # Add marker layer ontop of heat map

# marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info, 
#                                  fill_color='rgba(0, 150, 0, 0.4)', 
#                                  stroke_color='rgba(0, 0, 150, 0.4)', 
#                                  scale=5 )


# fig = gmaps.figure()

# fig.add_layer(marker_layer)
# fig.add_layer(heat_layer)


# fig

# # Display figure


TypeError: marker_layer() got an unexpected keyword argument 'fill_color'

In [23]:
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)


fig = gmaps.figure()

fig.add_layer(marker_layer)
fig.add_layer(heat_layer)


fig



Figure(layout=FigureLayout(height='420px'))